In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.svm import SVC
def normalation(X_train, X_test, y_train, y_test):
#define scaler
    scaler ={
        "StandardScaler": StandardScaler(),
        "MinMaxScaler": MinMaxScaler(),
        "RobustScaler":RobustScaler(),
        "Normalizer": Normalizer()
    }

    param_grid = {'svc__C':[0.01,0.1,1,10,100], 'svc__gamma':[0.001,0.01,0.1,1]}


    test_error_rate = 10
    #loop through each scaler, create a piperline. 
    for scaler_name, scaler in scaler.items():
        pipe = Pipeline([("scaler", scaler),("svc",SVC())])

        grid = GridSearchCV(pipe, param_grid=param_grid,cv=5,n_jobs=2)
        #grid.fit(X_train_Wine, y_train_Wine)
        grid.fit(X_train,y_train)
        model = grid.best_estimator_
        kf = KFold(n_splits=5, shuffle=True, random_state=2404)
        p_values = []

         # Conformity computation for cross-conformal prediction
        for train_index, calibration_index in kf.split(X_train):
            # Split into training and calibration sets
            X_train_cv, X_calib = X_train[train_index], X_train[calibration_index]
            y_train_cv, y_calib = y_train[train_index], y_train[calibration_index]


            # Fit model on training fold
            model.fit(X_train_cv, y_train_cv)
    
            # Compute conformity scores using decision_function
            calib_scores = model.decision_function(X_calib)
            calib_pred = np.max(calib_scores, axis=1)
    
            # Evaluate on test set
            test_scores = model.decision_function(X_test)
            test_pred = np.max(test_scores, axis=1)
    
            # Compute p-values
            for i, score in enumerate(test_pred):
                p_value = np.mean(calib_pred <= score)
                p_values.append(p_value)

        #Convert p-values to numpy array
        p_values = np.array(p_values).reshape(-1, len(X_test))
        #p_values = np.array(p_values)

        # Compute the calibration curve
        eps = np.linspace(0, 1, 100)
        error_rate = []

        for e in eps:
            error_rate.append(np.mean(p_values < e))

        # Plot calibration curve
        plt.plot(eps, error_rate, label='Calibration Curve')
        plt.plot(eps, eps, '--', label='Ideal')
        plt.xlabel('Significance Level (ε)')
        plt.ylabel('Error Rate')
        plt.title('Calibration Curve')
        plt.legend()
        plt.grid()
        plt.show()

        #print( y_test)
        #print(model.predict(X_test))
        #p_values = np.array(p_values).reshape(-1, len(X_test))
        average_false_p_value = np.mean(p_values[:, y_test != model.predict(X_test)])
        # Compute the average false p-value
        #average_false_p_value = np.mean(p_values[y_test != model.predict(X_test)])
        print(f"Average False p-value on Test Set: {average_false_p_value}")

In [ ]:
#question 7b

from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.svm import SVC

def mormalization11(X_train, X_test, y_train, y_test):
    # Define scalers
    scalers = {
        "StandardScaler": StandardScaler(),
        "MinMaxScaler": MinMaxScaler(),
        "RobustScaler": RobustScaler(),
        "Normalizer": Normalizer()
    }

    

    for scaler_name, scaler in scalers.items():
        pipe = Pipeline([("scaler", scaler), ("svc", SVC(decision_function_shape='ovr'))])
        grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=2)
        #grid.fit(X_train, y_train)

        kf = KFold(n_splits=5, shuffle=True, random_state=2404)
        # Conformity computation for cross-conformal prediction
        for train_index, calibration_index in kf.split(X_train):
            # Split into training and calibration sets
            X_rest, X_fold = X_train[train_index], X_train[calibration_index]
            y_rest, y_fold = y_train[train_index], y_train[calibration_index]
            grid.fit(X_rest, y_rest)
            cal = grid.decision_function(X_test)



SyntaxError: invalid syntax (2677147537.py, line 8)

NameError: name 'normalation11' is not defined

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from scipy.stats import rankdata

def conformal_score(X_train, y_train, X_calibrate, y_calibrate, new_instance, model):
    # Train model
    model.fit(X_train, y_train)
    
    # Get decision function scores for the calibration set and new instance
    calibration_scores = model.decision_function(X_calibrate)
    new_instance_score = model.decision_function([new_instance])
    
    # Combine and rank scores
    all_scores = np.concatenate((calibration_scores, new_instance_score))
    ranks = rankdata(-all_scores, method='ordinal')  # We use 'negative' for higher scores to have higher ranks
    
    # Compute p-value for the new instance (last element in ranks array)
    p_value = (np.sum(ranks[-1] <= ranks[:-1]) + 1) / (len(all_scores) + 1)
    return p_value

# Load data
data = load_iris()
X, y = data.data, data.target

# Prepare K-fold
k = 5
kf = KFold(n_splits=k)

# SVM model
svm = SVC(kernel='linear')

# Select new instance
new_instance = X[100]  # Example instance

# Perform cross-conformal prediction
p_values = []
for train_index, calibrate_index in kf.split(X):
    X_train, X_calibrate = X[train_index], X[calibrate_index]
    y_train, y_calibrate = y[train_index], y[calibrate_index]
    
    # Ensure the new instance is not in the calibration set
    if new_instance in X_calibrate:
        continue
    
    p_value = conformal_score(X_train, y_train, X_calibrate, y_calibrate, new_instance, svm)
    p_values.append(p_value)

# Average p-values across all splits where the new instance was not used for calibration
average_p_value = np.mean(p_values)
print("Average p-value for the new instance:", average_p_value)


In [ ]:
i dont know it my the code just upload calculate properly the p-value

#question 7 
#this is my attemp to implement question 7. I nkeo that the implemantion is not right but i tried anyway.
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.svm import SVC

def kFolds(X_train, X_test, y_train, y_test):

    

    
#define scaler
    scaler ={
        "StandardScaler": StandardScaler(),
        "MinMaxScaler": MinMaxScaler(),
        "RobustScaler":RobustScaler(),
        "Normalizer": Normalizer()
    }

    param_grid = {'svc__C':[0.01,0.1,1,10,100], 'svc__gamma':[0.001,0.01,0.1,1]}


    test_error_rate = 10
    #loop through each scaler, create a piperline. 
    for scaler_name, scaler in scaler.items():
        pipe = Pipeline([("scaler", scaler),("svc",SVC())])
        #for i in range(5,11):
        grid = GridSearchCV(pipe, param_grid=param_grid,cv=5,n_jobs=2)
        #grid.fit(X_train_Wine, y_train_Wine)
        grid.fit(X_train,y_train)
        #print("best parameters: ", grid.best_params_)
        model = grid.best_estimator_
        kf = KFold(n_splits=5, shuffle=True, random_state=2404)
        p_values = []

        # Conformity computation for cross-conformal prediction
        for train_index, calibration_index in kf.split(X_train):
            # Split into training and calibration sets
            X_train_cv, X_calib = X_train[train_index], X_train[calibration_index]
            y_train_cv, y_calib = y_train[train_index], y_train[calibration_index]


            # Fit model on training fold
            model.fit(X_train_cv, y_train_cv)
    
            # Compute conformity scores  for each fold using decision_function
            conformity_scores = model.decision_function(X_calib)

            calib_pred = np.max(conformity_scores, axis=1)
            #print("calibration ",conformity_scores)
            #print(calib_pred)
            
            # Evaluate on test set
            test_scores = model.decision_function(X_test)

            test_pred = np.max(test_scores, axis=1)
            #print("test_predict", test_scores)
            for i in range(len(test_pred)):
                p_value = np.mean(calib_pred <= test_pred[i])
                p_values.append(p_value)

            # Compute p-values
            # for i, score in enumerate(test_pred):

            #     p_value = np.mean(calib_pred <= score)
            #     p_values.append(p_value)
        #Convert p-values to numpy array
        p_values = np.array(p_values).reshape(-1, len(X_test))
        #p_values = np.array(p_values)


        # Compute the calibration curve
        eps = np.linspace(0, 1, 100)
        error_rate = []
        
        for e in eps:
            #print(error_rate)
            error_rate.append(np.mean(p_values <= e))

        # Plot calibration curve
        plt.plot(eps, error_rate, label=f'{scaler_name}')
        plt.plot(eps, error_rate, label='Calibration Curve')
        plt.plot(eps, eps, '--', label='Ideal')
        plt.xlabel('Significance Level (ε)')
        plt.ylabel('Error Rate')
        plt.title('Calibration Curve')
        plt.legend()
        plt.grid()
        plt.show()

        #print( y_test)
        #print(model.predict(X_test))
        #p_values = np.array(p_values).reshape(-1, len(X_test))
        average_false_p_value = np.mean(p_values[:, y_test != model.predict(X_test)])
        # Compute the average false p-value
        #average_false_p_value = np.mean(p_values[y_test != model.predict(X_test)])
        print(f"Average False p-value on Test Set for : {scaler_name} {average_false_p_value}")

In [ ]:
#question 7 
#this is my attemp to implement question 7. I nkeo that the implemantion is not right but i tried anyway.
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.svm import SVC

def kFolds(X_train, X_test, y_train, y_test):

    

    
#define scaler
    scaler ={
        "StandardScaler": StandardScaler(),
        "MinMaxScaler": MinMaxScaler(),
        "RobustScaler":RobustScaler(),
        "Normalizer": Normalizer()
    }

    param_grid = {'svc__C':[0.01,0.1,1,10,100], 'svc__gamma':[0.001,0.01,0.1,1]}

    count = 0
    #test_error_rate = 10
    #loop through each scaler, create a piperline. 
    for scaler_name, scaler in scaler.items():
        pipe = Pipeline([("scaler", scaler),("svc",SVC())])
        #for i in range(5,11):
        grid = GridSearchCV(pipe, param_grid=param_grid,cv=5,n_jobs=2)
        #grid.fit(X_train_Wine, y_train_Wine)
        grid.fit(X_train,y_train)
        #print("best parameters: ", grid.best_params_)
        model = grid.best_estimator_
        kf = KFold(n_splits=5, shuffle=True, random_state=2904)
        p_values = []
        p_valuesrr = []

        # Conformity computation for cross-conformal prediction
        for rest_fold_index, fold_index in kf.split(X_train):
            # Split into training and calibration sets
            X_rest_fold, X_fold = X_train[rest_fold_index], X_train[fold_index]
            y_rest, y_fold = y_train[rest_fold_index], y_train[fold_index]


            # Fit model on training fold
            model.fit(X_rest_fold, y_rest)
    
            # Compute conformity scores  for each fold using decision_function
            conformity_scores = model.decision_function(X_fold)

            #retrieve max volues of of calibration set.
            calib_pred = np.max(conformity_scores, axis=1)
            #print("calibration ",conformity_scores)
            
            # Evaluate on test set
            test_scores = model.decision_function(X_test)

            #retrieve max volues of of calibration set.
            test_pred = np.max(test_scores, axis=1)


            #combine all ranks score
            ######
            #this fucntion user rank calcualtion
            #############
            for i in range(len(test_pred)):
                #print("test_pred",test_pred)

                ###############################
                #CHECK THE RANK
                #############################
                rank = np.sum(calib_pred <= test_pred[i]) #rank for each labels.
                #print("cont ",rank)
                #if i == len(test_pred):
                
                p_valueFUNCTION = ((rank) + 1) /(len(test_pred) + 1)
                p_valuesrr.append(p_valueFUNCTION)
            #print("p_valuerss", p_valuesrr)




            
            #originaL CODE   
            for i in range(len(test_pred)):
                #if i == len(test_pred):
                p_value = np.mean(calib_pred <= test_pred[i])
                p_values.append(p_value)
            #print("p_values ",len(p_values))

        p_values = np.array(p_values).reshape(-1, len(X_test))
        p_values_FUNCTION = np.array(p_valuesrr).reshape(-1, len(X_test))
        #print("p_values ",p_values)
        #print("p_valuerss", p_valuesSSSSSSSSS)
        #p_values = np.array(p_values)


        # Compute the calibration curve
        eps = np.linspace(0, 1, 100)
        error_rate = []
        
        for e in eps:
            #print(error_rate)
            error_rate.append(np.mean(p_values_FUNCTION <= e))

        # Plot calibration curve
        plt.plot(eps, error_rate, label=f'{scaler_name}')
        plt.plot(eps, error_rate, label='Calibration Curve')
        plt.plot(eps, eps, '--', label='Ideal')
        plt.xlabel('Significance Level (ε)')
        plt.ylabel('Error Rate')
        plt.title('Calibration Curve')
        plt.legend()
        plt.grid()
        plt.show()



        ##########################################################################
        average_false_p_valueN = calculate_average_false_p_value(model, X_test, y_test, p_values)
        print(f"Average False p-valueN on Test Set for : {scaler_name} {average_false_p_valueN}")


        #################################################################################
        
        average_false_p_value = np.mean(p_values[:, y_test != model.predict(X_test)])
        # Compute the average false p-value
        #average_false_p_value = np.mean(p_values[y_test != model.predict(X_test)])
        print(f"Average False p-value on Test Set for : {scaler_name} {average_false_p_value}")
        #print(f"Average False p-value on Test Set for : {scaler_name} {MY_AVERAGE_FALSE_P_VALUE}")


#function responsibe to calcualte the average false p-value.
def calculate_average_false_p_value(model, X_test, y_test, p_values):
    print(model)
    predictions = model.predict(X_test)
    incorrect_predictions = y_test != predictions
    
    p_values = np.array(p_values)
    if len(p_values.shape) == 2:
        p_values = np.mean(p_values, axis=0)
    
    false_p_values = p_values[incorrect_predictions]
    if false_p_values.size > 0:
        average_false_p_value = np.mean(false_p_values)
    else:
        average_false_p_value = np.nan  # or handle as appropriate

    return average_false_p_value

In [ ]:
'''

#question 7 
#this is my attemp to implement question 7. I nkeo that the implemantion is not right but i tried anyway.
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.svm import SVC

def kFolds(X_train, X_test, y_train, y_test):

    

    
#define scaler
    scaler ={
        "StandardScaler": StandardScaler(),
        "MinMaxScaler": MinMaxScaler(),
        "RobustScaler":RobustScaler(),
        "Normalizer": Normalizer()
    }

    param_grid = {'svc__C':[0.01,0.1,1,10,100], 'svc__gamma':[0.001,0.01,0.1,1]}

    count = 0
    #test_error_rate = 10
    #loop through each scaler, create a piperline. 
    for scaler_name, scaler in scaler.items():
        pipe = Pipeline([("scaler", scaler),("svc",SVC())])
        #for i in range(5,11):
        grid = GridSearchCV(pipe, param_grid=param_grid,cv=5,n_jobs=2)
        #grid.fit(X_train_Wine, y_train_Wine)
        grid.fit(X_train,y_train)
        #print("best parameters: ", grid.best_params_)
        model = grid.best_estimator_
        kf = KFold(n_splits=5, shuffle=True, random_state=2404)
        p_values = []
        p_valuesrr = []

        # Conformity computation for cross-conformal prediction
        for rest_fold_index, fold_index in kf.split(X_train):
            # Split into training and calibration sets
            X_rest_fold, X_fold = X_train[rest_fold_index], X_train[fold_index]
            y_rest, y_fold = y_train[rest_fold_index], y_train[fold_index]


            # Fit model on training fold
            model.fit(X_rest_fold, y_rest)
    
            # Compute conformity scores  for each fold using decision_function
            conformity_scores = model.decision_function(X_fold)

            #retrieve max volues of of calibration set.
            calib_pred = np.max(conformity_scores, axis=1)
            #print("calibration ",conformity_scores)
            
            # Evaluate on test set
            test_scores = model.decision_function(X_test)

            #retrieve max volues of of calibration set.
            test_pred = np.max(test_scores, axis=1)
            #print(test_pred)
            
            #originaL CODE   
            for i in range(len(test_pred)):
                #if i == len(test_pred):
                p_value = np.mean(calib_pred <= test_pred[i]) #rank for each labels.
                p_values.append(p_value)
            #print("p_values ",p_values)

        p_values = np.array(p_values).reshape(-1, len(X_test))
        #p_values_FUNCTION = np.array(p_valuesrr).reshape(-1, len(X_test))
        #print("p_values ",p_values)
        #print("p_valuerss", p_valuesSSSSSSSSS)
        #p_values = np.array(p_values)


        # Compute the calibration curve
        eps = np.linspace(0, 1, 100)
        error_rate = []
        
        for e in eps:
            #print(error_rate)
            error_rate.append(np.mean(p_values <= e))

        # Plot calibration curve
        plt.plot(eps, error_rate, label=f'{scaler_name}')
        plt.plot(eps, error_rate, label='Calibration Curve')
        plt.plot(eps, eps, '--', label='Ideal')
        plt.xlabel('Significance Level (ε)')
        plt.ylabel('Error Rate')
        plt.title('Calibration Curve')
        plt.legend()
        plt.grid()
        plt.show()



    
        average_false_p_valueN = calculate_average_false_p_value(model, X_test, y_test, p_values)
        print(f"Average False p-valueN on Test Set for {scaler_name} is {average_false_p_valueN}")


#function responsibe to calcualte the average false p-value.
def calculate_average_false_p_value(model, X_test, y_test, p_values):
    #print(model)
    predictions = model.predict(X_test)
    incorrect_predictions = y_test != predictions
    
    p_values = np.array(p_values)
    if len(p_values.shape) == 2:
        p_values = np.mean(p_values, axis=0)
        #print("shape p-valuie after averaging:",  p_values.shape)
    
    false_p_values = p_values[incorrect_predictions]
    #print("false__p_values.size ",false_p_values.size)
    if false_p_values.size > 0:
        average_false_p_value = np.mean(false_p_values)
    else:
        average_false_p_value = 0  # or handle as appropriate

    return average_false_p_value
'''